# This is NOT the main Script
## This notebook was used to help build the main script.
### Selenium Webscraping of Fidelity Stock Research

More information on how the notebook works will be added later.

In [10]:
# imports
from selenium.webdriver import Firefox
import time
from selenium.webdriver.common.by import By
import pandas as pd
import re

In [ ]:
# coming later; this cell, and likely the ones underneath can be used to read the csv data, create a new excel document to write to, and copy over the relevant data into the new excel document.

In [28]:
# open the browser, navigate to Fidelity.com
# this cell assumes that you have added the webdriver to your system path
driver = Firefox()
driver.get("https://www.fidelity.com/")

After the browser navigates to the Fidelity website, go to the browser and log in as you normally would. Once logged in, you can run the next cell.

In [ ]:
# in the final version of this, this is approximately where we would start loading pages and extracting data. During development, this will be a series of that accomplish individual tasks, which will be compiled together into a single script later.

In [12]:
# this loads the stock research page for V
driver.get("https://snapshot.fidelity.com/fidresearch/snapshot/landing.jhtml#/research?symbol=V&appCode=&optInNRE=")
# a lot of the elements on the Fidelity research pages won't load unless they are scrolled past in the browser. This next line automates that step. Thanks to pythonbasics.org for this trick.
time.sleep(10)
driver.execute_script("window.scrollTo(0,(document.body.scrollHeight / 2))")
time.sleep(4)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [5]:
# first, determine if the security is a stock, ETF, mutual fund, or other
# TO FIX: this does not work on Mutual Fund pages
try:
    sec_type = driver.find_element_by_class_name('breadcrumb-div')
except:
    try:
        print('must be a MF')
        sec_type = driver.find_element_by_xpath('/html/body/div[3]/div/div/div/header-template/div[1]/div[1]')
    except:
        print("Must not be a MF either, can't find anything")
try:
    sec_type = sec_type.text
    print(sec_type)
except AttributeError:
    pass
sec_type.strip("»")
sec_type = sec_type.split()
for x in sec_type:
    if x == 'Stocks':
        sec_type = x
        break
    elif x == 'ETFs':
        sec_type = x
        break
    elif x == 'Mutual':
        sec_type = 'Mutual Funds'
        break
sec_type

Research > Stocks >
Video Help/Glossary


'Stocks'

In [8]:
# analysis items for stocks
analysis_items = driver.find_element_by_class_name('analysis-items')
analysis_items.text

'Valuation\ncurrent =\n63\nOvervalued\n= 0\nUndervalued\n= 100\nQuality\ncurrent =\n50\nLow\n= 0\nHigh\n= 100\nGrowth Stability\ncurrent =\n11\nLow\n= 0\nHigh\n= 100\nFinancial Health\ncurrent =\n49\nLess Healthy\n= 0\nHealthy\n= 100'

In [45]:
# regex to extract useful data
valuation_pos = analysis_items.text.index('Valuation\ncurrent =')
valuation = analysis_items.text[(valuation_pos + 19) : (valuation_pos + 23)]
valuation = re.findall(r'\d+', valuation)
int(valuation[0])

63

In [47]:
# regex for Quality
quality_pos = analysis_items.text.index("Quality")
quality = analysis_items.text[(quality_pos + 18) : (quality_pos + 22)]
quality = re.findall(r'\d+', quality)
int(quality[0])

50

In [50]:
# regex for Growth Stability
growth_stability_pos = analysis_items.text.index("Growth")
growth_stability = analysis_items.text[(growth_stability_pos + 23) : (growth_stability_pos + 30)]
growth_stability = re.findall(r"\d+", growth_stability)
int(growth_stability[0])

11

In [51]:
# regex for Financial Health
fin_health_pos = analysis_items.text.index("Health")
fin_health = analysis_items.text[(fin_health_pos + 15): (fin_health_pos + 20)]
fin_health = re.findall(r"\d+", fin_health)
int(fin_health[0])

49

In [9]:
# equity summary score
equity_summ_score = driver.find_element_by_class_name('rating-score')
equity_summ_score.text

'4.6'

In [55]:
# 52 Week Performance
fifty_two_week = driver.find_elements_by_id('box14')
ftw_perf = fifty_two_week[0].text.splitlines()
float(ftw_perf[1].strip('+').strip('%'))

-456.69

In [58]:
# Div yield
dividends = driver.find_elements_by_id('box11')
dividends = dividends[0].text.split()
for item in dividends:
    if '$' in item:
        div_amt = float(item.strip('$'))
    if '%' in item:
        div_yield = float(item.strip('%'))
div_amt, div_yield

(1.28, 0.6)

In [70]:
# recognia short term outlook
# unsure how well this will work w/ other things
# the xpath may be a problematic way to do this
short_term = driver.find_elements_by_xpath('/html/body/div[3]/div[2]/div[1]/div[11]/div/div/div/div/section/div/div[1]/div[2]')
short_list = short_term[0].get_attribute('innerHTML').split()
for i in range(0, len(short_list)):
    if short_list[i] == 'bold':
        recognia_short_term = short_list[i + 1]
        break
recognia_short_term = re.findall(r'[a-z]*', recognia_short_term)
recognia_short_term[0]

'weak'

In [74]:
# attempting to apply same method to mid-term
# i should convert this to a function perhaps
mid_term = driver.find_elements_by_xpath('/html/body/div[3]/div[2]/div[1]/div[11]/div/div/div/div/section/div/div[2]')
mid_list = mid_term[0].get_attribute('innerHTML').split()
for i in range(0, len(mid_list)):
    if mid_list[i] == 'bold':
        recognia_mid_term = mid_list[i + 1]
        break
recognia_mid_term = re.findall(r'[a-z]*', recognia_mid_term)
recognia_mid_term[0]

'strong'

In [5]:
# attempting to apply same method to mid-term
# i should convert this to a function perhaps
long_term = driver.find_elements_by_xpath('/html/body/div[3]/div[2]/div[1]/div[11]/div/div/div/div/section/div/div[3]')
long_list = long_term[0].get_attribute('innerHTML').split()
for i in range(0, len(long_list)):
    if long_list[i] == 'bold':
        recognia_long_term = long_list[i + 1]
        break
recognia_long_term = re.findall(r'[a-z]*', recognia_long_term)
recognia_long_term[0]

'strong'

In [29]:
# next, now that we have all of the information we want from the first page, we move to the key stats page
driver.get('https://snapshot.fidelity.com/fidresearch/snapshot/landing.jhtml#/keyStatistics?symbol=V')

# then, we scroll thru the page like we did earlier to ensure all elements have rendered to the page and are thus accessible for us to extract
time.sleep(10)
driver.execute_script("window.scrollTo(0,(document.body.scrollHeight / 2))")
time.sleep(4)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [23]:
# NOTE: this will be deprecated; see the cell below, which works in a much more efficient way

# now, extract the necessary data; start w/ P/E ratio TTM
# the first number in each row describes the actual company statistic, the second is the industry average
key_stats = driver.find_elements_by_class_name('datatable-component')
key_stats = key_stats[0].text.splitlines()
# key_stats
for stat in key_stats:
    if 'P/E (Trailing Twelve Months)' in stat:  # P/E TTM
        pe_ttm = stat.split()
        for x in pe_ttm:
            if '.' in x:
                pe_ttm = float(x)   # we only want the first value that would meet this criteria; this would be the actual company's P/E, not the industry one
                break
    elif 'P/E (5-Year Average)' in stat:    # P/E 5 year average
        pe_five_yr = stat.split()
        for x in pe_five_yr:
            if '.' in x:
                pe_five_yr = float(x)
                break
    elif 'Price/Cash Flow (Most' in stat:   #P/CF most recent quarter
        pcf_quarter = stat.split()
        for x in pcf_quarter:
            if '.' in x:
                pcf_quarter = float(x)
                break
    elif 'Price/Cash Flow (TTM)' in stat:
        pcf_ttm = stat.split()
        for x in pcf_ttm:
            if '.' in x:
                pcf_ttm = float(x)
                break
pe_ttm, pe_five_yr, pcf_quarter, pcf_ttm


(43.88, 34.36, 34.15, 39.35)

In [32]:
# the above cell works, but I think there is likely a more efficient way to write it using dicts
stock_dict = dict()

# normally, the values would be added stock by stock, but since we're mainly developing using V, I will just manually add it
stock_dict['V'] = dict()

# now, we need a list of statistics that we are looking for to scrape. We're going to use a tuple, because we won't be changing it.
stats = ('P/E (Trailing Twelve Months)', 'P/E (5-Year Average)', 'Price/Cash Flow (Most Recent Quarter)', 'Price/Cash Flow (TTM)', 'Price/Sales (Most Recent Quarter)', 'Price/Sales (TTM)', 'Price/Book')

# extract data from the page
key_stats = driver.find_elements_by_class_name('datatable-component')
key_stats = key_stats[0].text.splitlines()

# iterate thru data and add to dict
# we would normally do this via iteration of all symbols
for key, val in stock_dict.items():
    for stat in key_stats:
        for x in stats:
            if x in stat:
                y = stat.split()
                for z in y:
                    if '.' in z:
                        val[x] = float(z)
                        break

# this next part is to test to see if the above code worked
for key,val in stock_dict.items():
    for x, y in val.items():
        print(x,y)

P/E (Trailing Twelve Months) 43.88
P/E (5-Year Average) 34.36
Price/Cash Flow (Most Recent Quarter) 34.15
Price/Cash Flow (TTM) 39.35
Price/Sales (Most Recent Quarter) 19.95
Price/Sales (TTM) 21.07
Price/Book 14.54


In [36]:
# conver the dict to a DF
# I think this may be the easiest/fastest way to do this given my data and my extraction methods

stock_df = pd.DataFrame.from_dict(stock_dict, orient='index')
stock_df

,P/E (Trailing Twelve Months),P/E (5-Year Average),Price/Cash Flow (Most Recent Quarter),Price/Cash Flow (TTM),Price/Sales (Most Recent Quarter),Price/Sales (TTM),Price/Book
V,43.88,34.36,34.15,39.35,19.95,21.07,14.54
